Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [x] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [x] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [x] Do one-hot encoding of categorical features.
- [x] Do feature selection with `SelectKBest`.
- [x] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [x] Get mean absolute error for the test set.
- [x] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

## Main assignment goals

### Use a subset of the data 
where BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS' and the sale price was more than 100 thousand and less than 2 million

In [0]:
# Restrict category and sale price range
df = df[(df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS') &
        (df['SALE_PRICE'] <= 2000000) &
        (df['SALE_PRICE'] >= 100000)]

In [8]:
# Reduce cardinality for NEIGHBORHOOD feature
before = len(df['NEIGHBORHOOD'].value_counts())

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

# You get different results if you run this before the previous code cell
print(f'Reduced neighborhood count from {before} to 11')

Reduced neighborhood count from 185 to 11


In [9]:
df['NEIGHBORHOOD'].value_counts()

OTHER                       2406
FLUSHING-NORTH                98
ST. ALBANS                    94
SPRINGFIELD GARDENS           85
SO. JAMAICA-BAISLEY PARK      82
QUEENS VILLAGE                76
GREAT KILLS                   75
SOUTH OZONE PARK              70
BAYSIDE                       61
LAURELTON                     60
FLUSHING-SOUTH                57
Name: NEIGHBORHOOD, dtype: int64

In [10]:
# Preview data
print(df.shape)
df.head()

(3164, 21)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019
61,4,QUEENS VILLAGE,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,01/01/2019
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461.0,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1,810000,01/02/2019
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,NaN,A1,469 E 25TH ST,NaN,11226.0,1.0,0.0,1.0,"4,000",2680.0,1899.0,1,A1,125000,01/02/2019
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,NaN,A5,5521 WHITTY LANE,NaN,11203.0,1.0,0.0,1.0,"1,710",1872.0,1940.0,1,A5,620000,01/02/2019


In [11]:
# Check null values
df.isnull().sum()

BOROUGH                              0
NEIGHBORHOOD                         0
BUILDING_CLASS_CATEGORY              0
TAX_CLASS_AT_PRESENT                 0
BLOCK                                0
LOT                                  0
EASE-MENT                         3164
BUILDING_CLASS_AT_PRESENT            0
ADDRESS                              0
APARTMENT_NUMBER                  3163
ZIP_CODE                             0
RESIDENTIAL_UNITS                    0
COMMERCIAL_UNITS                     0
TOTAL_UNITS                          0
LAND_SQUARE_FEET                     0
GROSS_SQUARE_FEET                    0
YEAR_BUILT                           0
TAX_CLASS_AT_TIME_OF_SALE            0
BUILDING_CLASS_AT_TIME_OF_SALE       0
SALE_PRICE                           0
SALE_DATE                            0
dtype: int64

In [0]:
# After filtering the building class and sale price outliers, all EASE-MENT 
# values are null and only one APARTMENT_NUMBER value is not null.  
# We can safely drop those columns
df = df.drop(labels=['EASE-MENT', 'APARTMENT_NUMBER'], axis=1)

In [0]:
# LAND_SQUARE_FEET should be numeric instead of a string
# strip commas and convert to int
df['LAND_SQUARE_FEET'] = df['LAND_SQUARE_FEET'].str.replace(',', '').astype(int)

In [14]:
# Numeric data info
df.describe()

,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,SALE_PRICE
count,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.0,3.164000e+03
mean,6908.597029,75.847029,11027.442162,0.987358,0.015803,1.003161,3110.752212,1469.718394,1943.639697,1.0,6.281899e+05
std,3964.333893,160.779187,482.591574,0.114537,0.127241,0.172362,1790.452745,586.645088,26.679176,0.0,2.969134e+05
min,21.000000,1.000000,10030.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1890.000000,1.0,1.000000e+05
25%,4003.250000,21.000000,10461.000000,1.000000,0.000000,1.000000,2000.000000,1144.000000,1925.000000,1.0,4.450000e+05
50%,6269.500000,42.000000,11235.000000,1.000000,0.000000,1.000000,2541.500000,1360.000000,1938.000000,1.0,5.650000e+05
75%,10206.250000,69.000000,11413.000000,1.000000,0.000000,1.000000,4000.000000,1683.000000,1955.000000,1.0,7.600000e+05
max,16350.000000,2720.000000,11697.000000,2.000000,2.000000,3.000000,18906.000000,7875.000000,2018.000000,1.0,2.000000e+06


### Do train/test split
Use data from January — March 2019 to train. Use data from April 2019 to test.

In [0]:
# Split data by SALE_DATE
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'])
cutoff = pd.to_datetime('April 2019')
train = df[df['SALE_DATE'] < cutoff]
test = df[df['SALE_DATE'] >= cutoff]

### Do one-hot encoding of categorical features

In [16]:
# Categorical features
train.describe(exclude='number')

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_DATE
count,2517,2517,2517,2517,2517,2517,2517,2517
unique,5,11,1,2,13,2507,11,68
top,4,OTHER,01 ONE FAMILY DWELLINGS,1,A1,216-29 114TH ROAD,A1,2019-01-31 00:00:00
freq,1209,1918,2517,2486,921,2,921,78
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-01 00:00:00
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-30 00:00:00


In [20]:
# Decide which features to keep/encode
target = 'SALE_PRICE'
high_cardinality = ['ADDRESS', 'SALE_DATE']
features = train.columns.drop([target] + high_cardinality)
print(features)

# Set up X and y data with those features
X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
       'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT', 'BUILDING_CLASS_AT_PRESENT',
       'ZIP_CODE', 'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
       'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
       'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_TIME_OF_SALE'],
      dtype='object')


In [0]:
import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names=True)

In [0]:
# Encode X data
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

### Do feature selection with SelectKBest

In [31]:
# I was getting errors due to columns that had all the same value
# This finds those columns (all columns where std is 0)
X_train.describe().T[X_train.describe().T['std'] == 0]

,count,mean,std,min,25%,50%,75%,max
BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS,2517.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
TAX_CLASS_AT_TIME_OF_SALE,2517.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0


In [0]:
# Ok, I probably should have seen the building class coming, but tax class is 
# a good find.  Drop them both
X_train = X_train.drop(['BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS', 
                        'TAX_CLASS_AT_TIME_OF_SALE'], axis=1)
X_test = X_test.drop(['BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS', 
                      'TAX_CLASS_AT_TIME_OF_SALE'], axis=1)

In [0]:
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

k_selections = [] # will store [[k, mae]]

for k in range(1, len(X_train.columns)+1):
  # Set up selector with k columns
  selector = SelectKBest(score_func=f_regression, k=k)

  # Fit and transform
  X_train_selected = selector.fit_transform(X_train, y_train)
  X_test_selected = selector.transform(X_test)

  # Run linear regression and calculate error
  model = LinearRegression()
  model.fit(X_train_selected, y_train)
  y_pred = model.predict(X_test_selected)
  mae = mean_absolute_error(y_test, y_pred)

  # Save results
  k_selections.append([k, mae, X_train.columns[selector.get_support()]])

# converting makes it easier to use in visualizations
k_selections = pd.DataFrame(k_selections, columns=['k', 'mae', 'features'])

In [59]:
import plotly.express as px
px.bar(x=k_selections['k'], y=k_selections['mae'], range_y=[155000, 190000], 
       labels={'y':'Mean Absolute Error', 'x':'k features used'})

In [63]:
# features in that best result (row = k-1)
k_selections.iloc[10,2]

Index(['BOROUGH_3', 'BOROUGH_2', 'BOROUGH_5', 'NEIGHBORHOOD_FLUSHING-NORTH',
       'BLOCK', 'BUILDING_CLASS_AT_PRESENT_A3', 'ZIP_CODE', 'TOTAL_UNITS',
       'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A3'],
      dtype='object')

### Fit a ridge regression model with multiple features. 
Use the normalize=True parameter (or do feature scaling beforehand — use the scaler's fit_transform method with the train set, and the scaler's transform method with the test set)

In [130]:
from sklearn.linear_model import Ridge

alphas = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0]

# use features found during SelectKBest
features = list(k_selections.iloc[10,2])
print('Testing ridge regression with features', features)
for a in alphas:
  # Fit the model
  ridge = Ridge(alpha=a, normalize=True)
  ridge.fit(X_train[features], y_train)

  # Predict and get error
  y_pred = ridge.predict(X_test[features])
  mae = mean_absolute_error(y_test, y_pred)
  
  print(f'Alpha = {a}\tmae = {mae:.2f}')

Testing ridge regression with features ['BOROUGH_3', 'BOROUGH_2', 'BOROUGH_5', 'NEIGHBORHOOD_FLUSHING-NORTH', 'BLOCK', 'BUILDING_CLASS_AT_PRESENT_A3', 'ZIP_CODE', 'TOTAL_UNITS', 'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'BUILDING_CLASS_AT_TIME_OF_SALE_A3']
Alpha = 0.0001	mae = 159537.71
Alpha = 0.001	mae = 159501.99
Alpha = 0.01	mae = 159582.17
Alpha = 0.1	mae = 160490.27
Alpha = 1.0	mae = 173712.01
Alpha = 10.0	mae = 204342.48
Alpha = 100.0	mae = 213107.16


In [131]:
# RidgeCV does exactly what the loop above does, and returns the best result
from sklearn.linear_model import RidgeCV

# use features found during SelectKBest
features = list(k_selections.iloc[10,2])

# Fit the model
alphas = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
ridge = RidgeCV(alphas=alphas, normalize=True)
ridge.fit(X_train[features], y_train)

# Predict and get error
y_pred = ridge.predict(X_test[features])
mae = mean_absolute_error(y_test, y_pred)
  
print(f'Best alpha = {ridge.alpha_}\tmae = {mae:.2f}')

Best alpha = 0.001	mae = 159501.99


## Stretch Goals

### Feature engineering

In [0]:
train = df[df['SALE_DATE'] < cutoff]
test = df[df['SALE_DATE'] >= cutoff]

In [0]:
# Drop those columns that I already found are uniform throughout the data
# (building class because we filtered by that, tax class was coincidental)
train = train.drop(['TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_CATEGORY'], 
                   axis=1)
test = test.drop(['TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_CATEGORY'], 
                 axis=1)

In [0]:
# Total square feet column
train['TOTAL_SQ_FT'] = train['GROSS_SQUARE_FEET'] + train['LAND_SQUARE_FEET']
test['TOTAL_SQ_FT'] = test['GROSS_SQUARE_FEET'] + test['LAND_SQUARE_FEET']

In [0]:
# Building age
train['AGE'] = 2018 - train['YEAR_BUILT']
test['AGE'] = 2018 - test['YEAR_BUILT']

In [347]:
lot_block = (train['LOT']*5) + train['BLOCK']
lot_block.corr(train['SALE_PRICE'])

-0.152877946710259

In [0]:
# Seems random, but that's higher correlation than some of the 
# others we end up using
train['LOT_BLOCK'] = (train['LOT']*5) + train['BLOCK']
test['LOT_BLOCK'] = (test['LOT']*5) + test['BLOCK']

In [349]:
train.describe()

,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,SALE_PRICE,TOTAL_SQ_FT,AGE,LOT_BLOCK
count,2517.000000,2517.000000,2517.000000,2517.000000,2517.000000,2517.000000,2517.000000,2517.000000,2517.000000,2.517000e+03,2517.000000,2517.000000,2517.000000
mean,6750.859754,75.674215,10993.891538,0.987684,0.016289,1.003973,3146.558602,1473.500596,1944.729837,6.210113e+05,4620.059197,73.270163,7129.230830
std,3977.126707,157.232723,494.054748,0.110315,0.129712,0.171453,1798.698890,599.132016,27.054972,2.950794e+05,2104.425303,27.054972,4091.654791
min,21.000000,1.000000,10301.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1890.000000,1.000000e+05,0.000000,0.000000,106.000000
25%,3818.000000,21.000000,10314.000000,1.000000,0.000000,1.000000,2000.000000,1144.000000,1925.000000,4.400000e+05,3376.000000,58.000000,4099.000000
50%,6021.000000,42.000000,11234.000000,1.000000,0.000000,1.000000,2600.000000,1367.000000,1940.000000,5.600000e+05,4123.000000,78.000000,6459.000000
75%,9879.000000,70.000000,11413.000000,1.000000,0.000000,1.000000,4000.000000,1683.000000,1960.000000,7.500000e+05,5380.000000,93.000000,10278.000000
max,16350.000000,2720.000000,11697.000000,1.000000,2.000000,3.000000,18906.000000,7875.000000,2018.000000,2.000000e+06,25375.000000,128.000000,22476.000000


In [350]:
train.describe(exclude='number')

,BOROUGH,NEIGHBORHOOD,TAX_CLASS_AT_PRESENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_DATE
count,2517,2517,2517,2517,2517,2517,2517
unique,5,11,2,13,2507,11,68
top,4,OTHER,1,A1,216-29 114TH ROAD,A1,2019-01-31 00:00:00
freq,1209,1918,2486,921,2,921,78
first,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-01 00:00:00
last,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-30 00:00:00


### Re-run code to see if we got a better result

In [351]:
# Decide which features to keep/encode
# Also dropping class at present because since we want to know sale price, 
# we should look at class at sale
target = 'SALE_PRICE'
cols_to_drop = ['ADDRESS', 'SALE_DATE', 'BUILDING_CLASS_AT_PRESENT']
features = train.columns.drop([target] + cols_to_drop)
print(features)

# Set up X and y data with those features
X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

Index(['BOROUGH', 'NEIGHBORHOOD', 'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT',
       'ZIP_CODE', 'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
       'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
       'BUILDING_CLASS_AT_TIME_OF_SALE', 'TOTAL_SQ_FT', 'AGE', 'LOT_BLOCK'],
      dtype='object')


In [352]:
# One hot encode the X data
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

print(f'{len(X_train.columns)} columns after encoding')

41 columns after encoding


In [353]:
# SelectKBest loop
k_selections = [] # will store [[k, mae]]

for k in range(1, len(X_train.columns)+1):
  # Set up selector with k columns
  selector = SelectKBest(score_func=f_regression, k=k)

  # Fit and transform
  X_train_selected = selector.fit_transform(X_train, y_train)
  X_test_selected = selector.transform(X_test)

  # Run linear regression and calculate error
  model = LinearRegression()
  model.fit(X_train_selected, y_train)
  y_pred = model.predict(X_test_selected)
  mae = mean_absolute_error(y_test, y_pred)

  # Save results
  k_selections.append([k, mae, X_train.columns[selector.get_support()]])

# converting makes it easier to use in visualizations
k_selections = pd.DataFrame(k_selections, columns=['k', 'mae', 'features'])

px.bar(x=k_selections['k'], y=k_selections['mae'], range_y=[155000, 190000], 
       labels={'y':'Mean Absolute Error', 'x':'k features used'},
       title='SelectKBest with Linear Regression')

In [354]:
# SelectKBest loop with ridge regression instead
alphas = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
k_selections = [] # will store [[k, mae]]

for k in range(1, len(X_train.columns)+1):
  # Set up selector with k columns
  selector = SelectKBest(score_func=f_regression, k=k)

  # Fit and transform
  X_train_selected = selector.fit_transform(X_train, y_train)
  X_test_selected = selector.transform(X_test)

  # Run linear regression and calculate error
  model = RidgeCV(alphas=alphas, normalize=True)
  model.fit(X_train_selected, y_train)
  y_pred = model.predict(X_test_selected)
  mae = mean_absolute_error(y_test, y_pred)

  # Save results
  k_selections.append([k, mae, X_train.columns[selector.get_support()]])

# converting makes it easier to use in visualizations
k_selections = pd.DataFrame(k_selections, columns=['k', 'mae', 'features'])

px.bar(x=k_selections['k'], y=k_selections['mae'], range_y=[155000, 190000], 
       labels={'y':'Mean Absolute Error', 'x':'k features used'},
       title='SelectKBest with Ridge Regression')

In [355]:
# features in that best result
features = k_selections.iloc[11,2]
features

Index(['BOROUGH_3', 'BOROUGH_2', 'BOROUGH_5', 'NEIGHBORHOOD_FLUSHING-NORTH',
       'BLOCK', 'ZIP_CODE', 'TOTAL_UNITS', 'LAND_SQUARE_FEET',
       'GROSS_SQUARE_FEET', 'BUILDING_CLASS_AT_TIME_OF_SALE_A3', 'TOTAL_SQ_FT',
       'LOT_BLOCK'],
      dtype='object')

In [356]:
# So SelectKBest is actually the same method I was trying to do 
# manually in the last two assignments
test = X_train.copy()
test['PRICE'] = y_train
test.corr()['PRICE'].abs().sort_values(ascending=False).head(15)

PRICE                                1.000000
GROSS_SQUARE_FEET                    0.453373
TOTAL_SQ_FT                          0.360034
LAND_SQUARE_FEET                     0.270215
BUILDING_CLASS_AT_TIME_OF_SALE_A3    0.242734
BOROUGH_3                            0.224578
ZIP_CODE                             0.203109
BOROUGH_5                            0.172516
BOROUGH_2                            0.163513
LOT_BLOCK                            0.152878
BLOCK                                0.148242
TOTAL_UNITS                          0.147565
NEIGHBORHOOD_FLUSHING-NORTH          0.144310
COMMERCIAL_UNITS                     0.121620
BUILDING_CLASS_AT_TIME_OF_SALE_A5    0.114619
Name: PRICE, dtype: float64

### Result

I tried several things, but only managed to get minimum error down to 159,490.  (From the original 159,501)

I feel like the columns I ended up with are redundant too, there has to be something else I could try...